# PublicDatasets (Analyser)

## 1. Re-reading

In [1]:
special_separator = '___'

In [2]:
import pandas as pd
import re

In [3]:
research_paper_table = pd.read_csv('data/ResearchPapers.csv',index_col=0)

In [4]:
#Drop ignored
research_paper_table
research_paper_table = research_paper_table[research_paper_table.included] 

In [19]:
research_paper_table.head()

,Venue,Title,included
1,CHIL 2021,Data Augmentation for Electrocardiograms,True
2,CHIL 2021,Disability prediction in multiple sclerosis us...,True
3,CHIL 2021,Practical Challenges in Differentially-Private...,True
4,CHIL 2021,MedMCQA: A Large-scale Multi-Subject Multi-Cho...,True
5,CHIL 2021,Lead-agnostic Self-supervised Learning for Loc...,True


### 1.a get file names

In [6]:
def get_text_filename(v,t,path='data/texts/'):
    return path+v+special_separator+t+'.txt'

In [7]:
texts = []
for index, row in research_paper_table.iterrows():
    texts.append(get_text_filename(row["Venue"],row["Title"]))

In [8]:
# mention_matches


## 2. Get Dataset Mentions

### 2.1 Get occurences in datasets used in "Data and Code Availability"

In [9]:
def get_occurences_in_dasa(contents, low_name, before='', preview_offset = 100):
    idx0=contents.find("data and code availability")
    contents=contents[idx0:]
    idx=contents.find(low_name)
    if idx==-1: return 0 
    idx_end = idx + len(low_name)
    L = len(contents)
    start_edge = max(0,idx-preview_offset)
    end_edge = min(L-1,idx+preview_offset)
    print("\33[32mFound", name, "in", text_file,":\33[0m",'\n',
          contents[start_edge:idx]+'\033[5m\u001b[31m'+contents[idx:idx_end]+'\33[0m'+contents[idx_end:end_edge]
         )
    
def get_dasa(contents):
    """Get the Data and Code Availability section from CHIL papers"""
    idx0=contents.find("data and code availability")
    contents=contents[idx0:]
    idxend=contents.find('1. introduction')
    contents=contents[:idxend]
    return contents
    #print(contents)

In [10]:
#mention_matches = {name:[] for name in mentions}
items = {text_file:[] for text_file in texts}

#limit = 10
#i=0
for text_file in texts:
    #if i>limit: break
    with open(text_file, 'r') as f:
        contents = f.read()
        #Only check for 1-for-1 correspondence
        #AND DON'T FORGET TO LOWER CASE WHEN COMPARING!
        contents = contents.lower()
        #print(contents)

        items[text_file] = get_dasa(contents)
        #print('{}: ******************'.format(text_file))
        #low_name=name.lower()
        #if contents.find(low_name) != -1:
            #mention_matches[name].append(1)
        #    print(get_dasa(contents))
        #else:
            #mention_matches[name].append(0)
        #    pass
    #i+=1

In [11]:
def get_section(contents, section_header="data and code availability", section_end=['1. introduction','© 2022']):
    """Get the Data and Code Availability section from CHIL papers"""
    
    contents = contents.lower()
    contents = contents.replace("\n\n", "$$$" )
    contents = contents.replace("-\n", "" )
    # contents = contents.replace("\n", "" )
    contents = contents.replace("$$$", "\n" )

    idx0=contents.find(section_header)
    contents=contents[idx0:]
    for end in section_end:
        idxend=contents.find(end)
        if idxend==-1: continue
        contents=contents[:idxend]
        return contents
    #print(contents)

In [ ]:

get_dataset_section = get_section(contents, section_header="data and code availability", section_end=['1. introduction','© 2022']):"""Get the Data and Code Availability section from CHIL papers"""
#REPLACE THE get_dataset_section function to change behavior


In [12]:

items = {text_file:[] for text_file in texts}

for text_file in texts:
    with open(text_file, 'r') as f:
        contents = f.read()
        items[text_file] = get_section(contents)

## 4. WIP

In [13]:
#PATTERN 1: Citation
# ... [1-2][901]+[ab]*)
#
# In-line Citation
# r'[\)\.].* [1-2][901]+[ab]*)'

# pat = r'\(\D*\d{4}\)'
# Complete
# r'([a-zA-Z ]+[1-2][901]+[ab]*)'
def func1(str):
    pat = r'\([^\d)]*\d{4}(;\D*\d{4})*\)'
    return re.findall(pat,str)

In [14]:
#PATTERN 2: Footnote
#Footnote


def func2(str):
    pat = r'[1-9](?!=\))'
    return re.findall(pat,str)

In [15]:
#PATTERN 3: URL
#URL Pattern

#(https://urlregex.com/)
def func3(str):
    pat = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.findall(pat,str)

In [16]:
def collect_tokens(pat,str,name,tokens):
    matches = re.findall(pat,str)
    str = re.sub(pat, '', str)
    tokens |= {name: matches}
    return str

In [77]:
xn = r'\n?' #optional newline
son = r'(?: |\n)' #space or newline

author = r"(?:[A-Za-z'`-]+)"
etal = r"(?:et al\.?)"
additional = f"(?:{xn},?{son}(?:(?:and |& )?{author}|{etal}))"
year_num = "(?:19|20)[0-9][0-9]"
page_num = "(?:, p\.? [0-9]+)?"  # Always optional
year = fr"(?:, *{year_num}{page_num}| *\({year_num}{page_num}\))"
inline_citation = fr'\b(?!(?:Although|Also)\b){xn}{author}{xn}{additional}*{xn}{year}'
# ADAPTED FROM https://stackoverflow.com/a/63633049/2089784

num_section = r'([0-9]\.)+[0-9]'
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  #https://urlregex.com/
split_url_pattern = fr'h{xn}t{xn}t{xn}p{xn}[s]?{xn}:{xn}/{xn}/{xn}(?:[a-zA-Z{xn}]|[0-9]|[$-_@.&+]|[!*\(\{xn}),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

footnote_pattern = r'(?:(?<=[a-zA-Z\.]) ?|,)[1-9](?![\)\d])'

print(split_url_pattern)

h\n?t\n?t\n?p\n?[s]?\n?:\n?/\n?/\n?(?:[a-zA-Z\n?]|[0-9]|[$-_@.&+]|[!*\(\\n?),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+


In [78]:
def clean_newlines(dict):
    for key, val in dict.items():
        dict[key] = [ s.replace('\n','') for s in val ] 

In [81]:

# for name in mention_matches:
#     merged_dict[name]=mention_matches[name]
# for keyword in keyword_matches:
#     merged_dict[keyword]=keyword_matches[keyword]

# NEW CSV USING get_dasa function from 3.4

# for item in items:
#     print(item)
#     print(items[item])

for title, context in items.items():
    # print(func(pd.DataFrame( [ items[item] for item in items] ).iloc[i][0]))
    # print([ items[item] for item in items][i])
    # print(texts[i])
    # print(func1(pd.DataFrame( [ items[item] for item in items] ).iloc[i][0]))
    # print(func2(pd.DataFrame( [ items[item] for item in items] ).iloc[i][0]))
    # print(func3())
    
    references = {}
    # block = pd.DataFrame( [ items[item] for item in items] ).iloc[i][0]
    block = context
    
    if (type(block) == type(None)):
        print("ADD AN ERROR")
        continue
    
    block = collect_tokens(inline_citation,block,"In-line Citation",references)
    block = collect_tokens(num_section,block,"Numbered Section",{})
    block = collect_tokens(split_url_pattern,block,"URL",references)
    block = collect_tokens(footnote_pattern,block,"Footnote",references) 
    
    clean_newlines(references)
    
    print(references)
    
# texts[18]

# print(pd.DataFrame( [ items[item] for item in items] ).iloc[2][0])

# TODO: MAKE THESE INTO ROWS FOR THE CSV SHEET

{'In-line Citation': ['wagner et al., 2020', 'goldbergeret al., 2000'], 'URL': ['https://github.com/aniruddhraghu/ecg_aug.'], 'Footnote': []}
ADD AN ERROR
{'In-line Citation': [], 'URL': [], 'Footnote': [' 1']}
{'In-line Citation': [], 'URL': [], 'Footnote': []}
{'In-line Citation': ['reyna et al., 2021', 'wagner et al., 2020'], 'URL': ['https://github.com/jwoo5/fairseq-signals'], 'Footnote': ['1']}
{'In-line Citation': ['johnsonet al., 2019', 'irvin et al., 2019', 'johnson et al., 2021', 'goldberger et al., 2000', 'irvin et al., 2019'], 'URL': ['https://github.com/mlforhealth/cxr_fairness.'], 'Footnote': [' 1']}
{'In-line Citation': ['johnson et al., 2016', 'fivez et al., 2017', 'lu et al., 2019'], 'URL': ['https://github.com/dalgu90/cim-misspelling.'], 'Footnote': []}
{'In-line Citation': ['johnson et al., 2016', 'johnsonet al., 2016'], 'URL': ['https://github.com/sjpark9503/kg_txt_multimodal'], 'Footnote': ['1']}
{'In-line Citation': ['carnicelli et al., 2021'], 'URL': ['https://git

In [82]:
# #TODO:
# print("* SWITCH TO MULTI-MATCHING")